### Imports

In [1]:
from typing import TypedDict
print(TypedDict)

<function TypedDict at 0x105428c20>


In [2]:
import polars as pl
from great_tables import GT
import numpy as np
import gc

In [3]:
import altair as alt
# import plotly.express as px

#### Import Panel Data

In [4]:
grocery_df = pl.scan_csv(source="data/panel-datasets/edible_grocery.csv", 
                         has_header=True, 
                         separator=",", 
                         schema_overrides={'panel_id': pl.Int32,
                                           'trans_id': pl.Int32,
                                           'week': pl.Int16,
                                           'sku_id': pl.Categorical,
                                           'units': pl.Int16,
                                           'price': pl.Float32,
                                           'brand': pl.Categorical})

In [5]:
sku_df = pl.scan_csv(source="data/panel-datasets/sku_weight.csv", 
                         has_header=True, 
                         separator=",", 
                         schema_overrides={'sku_id': pl.Int32,
                                           'weight': pl.Int16})

In [6]:
kiwi_df = pl.scan_csv(source="data/panel-datasets/kiwibubbles_trans.csv", 
                      has_header=True, 
                      separator=",", 
                      schema_overrides={'ID': pl.Int16,
                                        'Market': pl.Categorical,
                                        'Day': pl.Int16,
                                        'Units': pl.Int16})

### Preliminaries

#### Weekly Sales Pattern

In [7]:
grocery_df = grocery_df.with_columns(
    ((pl.col('units') * pl.col('price')).cast(pl.Float64)).alias('spend')
)

result = (
    grocery_df
    .group_by(["week", "brand"])
    .agg(
        pl.col("spend").sum().alias('Total Spend')
    )
)

pivot_result = result.collect().pivot(
    on="brand",       # Columns (unique brands)
    values="Total Spend",  # Values to pivot
    index="week",          # Rows (index)
    aggregate_function=None  # No further aggregation since we've already summed
).sort("week")

sorted_columns = ["week"] + sorted([col for col in pivot_result.columns if col != "week"])
pivot_result = pivot_result.select(sorted_columns)
pivot_result

week,Alpha,Bravo,Charlie,Delta,Other
i16,f64,f64,f64,f64,f64
1,331.459999,247.780003,51.79,17.64,18.75
2,567.249997,398.640007,45.43,34.19,23.23
3,497.969999,639.980008,51.11,39.39,14.55
4,1389.960004,472.500004,51.87,85.04,39.779999
5,358.239999,252.540003,40.21,69.54,45.93
…,…,…,…,…,…
100,692.480006,420.479996,69.600001,54.590001,47.97
101,265.770002,264.659999,64.030001,35.05,26.88
102,379.610005,456.78999,43.5,20.4,43.61
